In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from plot_utils import plot_surface
import QAOA_paper_proxy

%matplotlib widget

def test_plot_number_with_cost(num_constraints: int, num_qubits: int) -> None:
    costs = np.linspace(0, num_constraints, num_constraints+1)
    num_with_costs = np.zeros(len(costs))
    for i in range (len(costs)):
        num_with_costs[i] = QAOA_paper_proxy.number_with_cost_paper_proxy(costs[i], num_constraints, num_qubits)
    
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(costs, num_with_costs)
    
def test_plot_by_cost_2(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_paper_proxy.number_of_costs_at_distance_paper_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from paper", "cost_2", "distance", "proxy")

test_plot_number_with_cost(20, 10)
test_plot_by_cost_2(4, 50, 12)
plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(10, 6, gammas, betas)
plot_heat_map(gammas, betas, overlaps, "Expectation Proxy from paper", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_proxy import QAOA_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray):
    expectations = np.zeros([len(gammas), len(betas)])
    amplitudes = np.zeros([num_constraints + 1, len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            amps, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)
            for c in range(num_constraints+1):
                amplitudes[c][i][j] = abs(amps[1][c]) ** 2

    return amplitudes, expectations


gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
amplitudes, overlaps = collect_parameter_data(21, 10, gammas, betas)

for c in range(0, 21):
    plot_heat_map(gammas, betas, amplitudes[c], f"Amplitude for c={c}", "Gamma", "Beta")

plot_heat_map(gammas, betas, overlaps, f"Expectation Proxy", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray):
    expectations = np.zeros([len(gammas), len(betas)])
    amplitudes = np.zeros([num_constraints + 1, len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            amps, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)
            for c in range(num_constraints+1):
                amplitudes[c][i][j] = abs(amps[1][c]) ** 2

    return amplitudes, expectations


gammas = np.linspace(0, np.pi/2, 25)
betas = np.linspace(0, np.pi, 25)
amplitudes, overlaps = collect_parameter_data(21, 10, gammas, betas)

for c in range(0, 21):
    plot_heat_map(gammas, betas, amplitudes[c], f"Amplitude for c={c}", "Gamma", "Beta")

plot_heat_map(gammas, betas, overlaps, f"Expectation Proxy", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_simulator import get_simulator, get_overlap, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)

    return overlaps

N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, overlaps, "True QAOA Overlap", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations

N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, overlaps, "True Expectation", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_overlap, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)


    return expectations, overlaps

N = 15
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
expectations, overlaps = collect_parameter_data(G, gammas, betas)
shifted_expectations = expectations - np.amin(expectations)
_ = plot_heat_map(gammas, betas, expectations, "True Expectation", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, overlaps, "True Overlap", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, (shifted_expectations/np.amax(shifted_expectations)) - (overlaps/np.amax(overlaps)), "True Expectation Minus Overlap", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_overlap, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np
from QAOA_proxy import QAOA_proxy
from QAOA_paper_proxy import QAOA_paper_proxy


def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations, overlaps


def collect_parameter_data_proxy_less_dumb(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray, h: float, hc: float, l: float, r: float) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    altered_proxy = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits, h, hc, l, r)
            _, altered_proxy[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits, h, hc, l, r, 2*(num_constraints+1)//3)

    return expectations, altered_proxy


def collect_parameter_data_proxy(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits, 2*(num_constraints+1)//3)

    return expectations


N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 10 edges
gammas = np.linspace(0, np.pi, 50)
betas = np.linspace(0, 2*np.pi, 50)
expectations, overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, expectations, "True QAOA Expectations", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, overlaps, "True QAOA Overlap", "Gamma", "Beta")
#expectation_proxies = collect_parameter_data_proxy(G.number_of_edges(), N, gammas, betas)
#_ = plot_heat_map(gammas, betas, expectation_proxies, "Expectation Proxies from Paper", "Gamma", "Beta")
expectation_proxies_less_dumb, altered_less_dumb = collect_parameter_data_proxy_less_dumb(G.number_of_edges(), N, gammas, betas, 0, 0, 1, 1)
_ = plot_heat_map(gammas, betas, expectation_proxies_less_dumb, "Expectation Proxies from us", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, altered_less_dumb, "Altered Proxies from us", "Gamma", "Beta")

In [ ]:
import QAOA_proxy as qpld
import numpy as np


def test_plot(num_constraints: int, num_qubits: int) -> None:
    costs = np.linspace(0, num_constraints, num_constraints + 1)
    num_with_costs = np.zeros(len(costs))
    for i in range(len(costs)):
        num_with_costs[i] = qpld.number_with_cost_proxy(costs[i], num_constraints, num_qubits)

    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(costs, num_with_costs)


test_plot(20, 10)

In [ ]:
import QAOA_proxy
import numpy as np
from plot_utils import plot_surface, animate_3d_rotation
import QAOA_paper_proxy
import matplotlib.animation as animation

%matplotlib widget

def test_plot_by_cost_paper(cost: int, num_constraints: int, num_qubits: int):
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_paper_proxy.number_of_costs_at_distance_paper_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    return plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from paper", "cost_2", "distance", "proxy")

def test_plot_by_cost(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_proxy.number_of_costs_at_distance_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    return plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from us", "cost_2", "distance", "proxy")

fig, ax = test_plot_by_cost_paper(8, 20, 10)
fig2, ax2 = test_plot_by_cost(8, 20, 10)

animate_3d_rotation(fig, ax, 'rotation1.gif')
animate_3d_rotation(fig2, ax2, 'rotation2.gif')

In [ ]:
from QAOA_paper_proxy import number_of_costs_at_distance_paper_proxy
import numpy as np
from plot_utils import plot_3d_scatter

%matplotlib widget

def test_plot(num_constraints_max: int, num_qubits_max: int) -> None:
    num_constraints = np.linspace(0, num_constraints_max, num_constraints_max + 1, dtype=int)
    num_qubits = np.linspace(0, num_qubits_max, num_qubits_max + 1, dtype=int)
    peak_hs = np.zeros([len(num_constraints), len(num_qubits)])
    for c in range(len(num_constraints)):
        for q in range(len(num_qubits)):
            try:
                peak_hs[c][q] = number_of_costs_at_distance_paper_proxy(c//2, c//2, q//2, c, q)
            except:
                ()
    
    plot_3d_scatter(num_constraints[7:], num_qubits, peak_hs[7:], f"Peak values from Paper", "num_constraints", "num_qubits", "peak value")

test_plot(15, 12)

In [ ]:
import numpy as np
from plot_utils import plot_3d_scatter

%matplotlib widget

def test_plot(num_constraints_max: int, num_qubits_max: int) -> None:
    num_constraints = np.linspace(0, num_constraints_max, num_constraints_max + 1, dtype=int)
    num_qubits = np.linspace(0, num_qubits_max, num_qubits_max + 1, dtype=int)
    peak_hs = np.zeros([len(num_constraints), len(num_qubits)])
    for c in range(len(num_constraints)):
        for q in range(len(num_qubits)):
            try:
                peak_hs[c][q] = 2**(q-4)
            except:
                ()
    plot_3d_scatter(num_constraints, num_qubits, peak_hs, f"Peak values from us", "num_constraints", "num_qubits", "peak value_proxy")

test_plot(15, 12)

In [ ]:
import networkx as nx

N = 10
nx.erdos_renyi_graph(N, 0.5, seed=42).number_of_edges()

In [1]:
from QAOA_simulator import get_expectation, get_simulator, get_overlap, QAOA_run
import networkx as nx
import qokit.maxcut as mc
import numpy as np
import QAOA_proxy as qp
import QAOA_proxy_optimizer as qpo
#import QAOA_paper_proxy as qpp
from itertools import product


N = 10
p = 2
G = nx.erdos_renyi_graph(N, 0.5, seed = 42)
num_edges = G.number_of_edges()
ising_model = mc.get_maxcut_terms(G)
sim = get_simulator(N, ising_model)
print(f"Max Cut: {max(sim.get_cost_diagonal())}")
init_gamma, init_beta = np.full((2, p), 0.1)

real_result = QAOA_run(ising_model, N, p, init_gamma, init_beta)
real_gamma = real_result["gamma"]
real_beta = real_result["beta"]
real_expectation = real_result["expectation"]
real_overlap = real_result["overlap"]
print(f"g: {real_gamma}, b: {real_beta}, e: {real_expectation}, o: {real_overlap}")

pre_result = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta)
pre_gamma = pre_result["gamma"]
pre_beta = pre_result["beta"]
pre_expectation = get_expectation(N, ising_model, pre_gamma, pre_beta)
pre_overlap = get_overlap(N, ising_model, pre_gamma, pre_beta)
print(f"g: {pre_gamma}, b: {pre_beta}, e: {pre_expectation}, o: {pre_overlap}")

init_h_tweaks = [1]
init_hc_tweaks = [0, 2]
init_l_tweaks = [0.1, 1, 2]
init_r_tweaks = [0.1, 1, 2]
init_tweaks = product(init_h_tweaks, init_hc_tweaks, init_l_tweaks, init_r_tweaks)

h, hc, l, r = qpo.QAOA_proxy_optimize(num_edges, N, p, init_gamma, init_beta, init_tweaks=init_tweaks)
print(f"h: {h}\nhc: {hc}\nl: {l}\nr: {r}")
result = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h, hc_tweak_add=hc, l_tweak_mul=l, r_tweak_mul=r)
gamma = result["gamma"]
beta = result["beta"]
expectation = get_expectation(N, ising_model, gamma, beta)
approx_ratio = expectation/max(sim.get_cost_diagonal())
overlap = get_overlap(N, ising_model, gamma, beta)
print(f"g: {gamma}, b: {beta}, e: {expectation}, o: {overlap}")

print("Start")
for _ in range(10):
    G = nx.erdos_renyi_graph(N, 0.5)
    while (G.number_of_edges() != num_edges):
        G = nx.erdos_renyi_graph(N, 0.5)
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)
    print(f"MaxCut: {max(sim.get_cost_diagonal())}")
    print(f"e: {get_expectation(N, ising_model, gamma, beta, sim)}, o: {get_overlap(N, ising_model, gamma, beta, sim)}")


#result2 = qpp.QAOA_paper_proxy_run(G.number_of_edges(), N, p, init_gamma, init_beta)
#gamma2 = result2["gamma"]
#beta2 = result2["beta"]
#expectation2 = get_expectation(N, ising_model, gamma2, beta2)
#print(f"g: {gamma2}, b: {beta2}, e: {expectation2}")

/home/nkohen/dev/QOKit/grips/QAOA_proxy.py:26: JuliaWarning: USE_JULIA=False, so calling python version of QAOA_proxy. Did you mean to call QAOA_proxy_python?
  warnings.warn("USE_JULIA=False, so calling python version of QAOA_proxy. Did you mean to call QAOA_proxy_python?", JuliaWarning)


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Max Cut: 15.0
g: [0.60729518 1.23338023 1.51375319], b: [0.4580484  0.3055645  0.16623899], e: 14.230756860040874, o: 0.5042338171708334
g: [ 0.73097805 -1.23440413 -0.16696553], b: [1.94927137 1.19545912 1.1864342 ], e: 8.160270598645209, o: 0.009908793618320278
h: 1.9682688294810495
hc: 2.749016430460764
l: 1.829672431219189
r: 1.2590326485080765
g: [-0.53786195 -0.07879607  0.87280154], b: [1.2226664  1.22464704 1.92107174], e: 12.764466884853158, o: 0.23203200693946163
Start
MaxCut: 15.0
e: 12.653307664275037, o: 0.1531939914474503
MaxCut: 16.0
e: 13.096239534551717, o: 0.15835161216874102
MaxCut: 16.0
e: 12.640448563126089, o: 0.08046690432958996
MaxCut: 17.0
e: 12.908129585173207, o: 0.07149563920101255
MaxCut: 17.0
e: 12.963392677437799, o: 0.07876652679623507
MaxCut: 16.0
e: 12.86229569696382, o: 0.05167833392757955
MaxCut: 16.0
e: 12.356335673639405, o: 0.049483823

In [1]:
import QAOA_proxy_optimizer as qpo

p = 2
min_N = 7
max_N = 11

results = qpo.find_optimal_parameters(min_N, max_N, p)
print(results)

/home/nkohen/dev/QOKit/grips/QAOA_proxy.py:26: JuliaWarning: USE_JULIA=False, so calling python version of QAOA_proxy. Did you mean to call QAOA_proxy_python?
  warnings.warn("USE_JULIA=False, so calling python version of QAOA_proxy. Did you mean to call QAOA_proxy_python?", JuliaWarning)


Starting work on N=7
Starting work on N=8
Starting work on N=9
Starting work on N=10
Starting work on N=11
{(7, 7): (1.1819564161988851, 0.29181379811852604, 0.47458492921390655, 0.5745897009614426), (7, 10): (2.396062910811212, 0.3944341933034466, 0.37526336165983853, 0.8252003916292558), (7, 13): (3.39536568380337, 0.4229929015872725, 0.29105151142603064, 0.8627876069339993), (7, 16): (5.690317305978456, 1.2420548536155116, 1.2381551020257706, 1.2277680416639836), (8, 9): (2.38379048264999, 1.1440185159703111, 1.4130909306419757, 1.7178078395828589), (8, 12): (3.184578035004384, 0.6406516244002661, 4.162870485391531, 2.840716591094424), (8, 15): (7.442948827697049, 0.04221335288332089, 5.568024655321806, 3.959909009643141), (8, 18): (9.102023630415966, 1.6976348883431414, 6.345171762031716, 4.67167025584174), (8, 21): (11.700856915177072, 2.3267822538370293, 7.507770206811433, 5.617263973542443), (8, 24): (14.753180434585984, 2.761127879120257, 2.4805715951413743, 1.8421037858200395)

In [2]:
regr, score = qpo.linear_regression_for_parameters(results)
print(regr.coef_)
print(score)
regr.predict([[8, 21]])

[[ 4.82918596  0.69330992]
 [-1.07668127  0.31519499]
 [ 1.93160494  0.12733136]
 [-0.72393632  0.19679883]]
0.8763526358743503


array([[12.39268908,  2.69256262,  4.87953122,  3.3152367 ]])

In [4]:
from QAOA_simulator import get_expectation, get_simulator, get_overlap
import QAOA_proxy as qp
import qokit.maxcut as mc
import Graph_util
import numpy as np

p = 2
init_gamma, init_beta = np.full((2, p), 0.1)

min_N = 7
max_N = 11

for N in range(min_N, max_N + 1):
    max_edges = N*(N-1)//2
    for num_edges in range(max_edges // 3, max_edges - 3, 3):
        h, hc, l, r = results[N, num_edges]
        res = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h, hc_tweak_add=hc, l_tweak_mul=l, r_tweak_mul=r)
        gamma = res["gamma"]
        beta = res["beta"]
        guess = regr.predict(np.array([[N, num_edges]]))
        h_guess, hc_guess, l_guess, r_guess = guess[0][0], guess[0][1], guess[0][2], guess[0][3]
        res_guess = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h_guess, hc_tweak_add=hc_guess, l_tweak_mul=l_guess, r_tweak_mul=r_guess)
        gamma_guess = res_guess["gamma"]
        beta_guess = res_guess["beta"]
        for _ in range(10):
            G = Graph_util.erdos_renyi(N, num_edges)
            ising_model = mc.get_maxcut_terms(G)
            sim = get_simulator(N, ising_model)
            max_cut = max(sim.get_cost_diagonal())
            optimal_e = get_expectation(N, ising_model, gamma, beta, sim)
            optimal_approx_ratio = optimal_e / max_cut
            guess_e = get_expectation(N, ising_model, gamma_guess, beta_guess, sim)
            guess_approx_ratio = guess_e / max_cut
            print(f"MaxCut ({N}, {num_edges}): {max_cut}")
            print(f"Optimal: e: {optimal_e}, o: {get_overlap(N, ising_model, gamma, beta, sim)}, ar: {optimal_approx_ratio}")
            print(f"Guess:   e: {guess_e}, o: {get_overlap(N, ising_model, gamma_guess, beta_guess, sim)}, ar: {guess_approx_ratio}")

MaxCut (7, 7): 6.0
Optimal: e: 5.063895070683046, o: 0.45121095025660085, ar: 0.8439825117805076
Guess:   e: 4.173159377038324, o: 0.2585650139265593, ar: 0.6955265628397207
MaxCut (7, 7): 7.0
Optimal: e: 5.189330449770927, o: 0.17396242484413282, ar: 0.7413329213958467
Guess:   e: 4.191414277372802, o: 0.07633252258338177, ar: 0.5987734681961145
MaxCut (7, 7): 6.0
Optimal: e: 4.859096534880548, o: 0.19209726587397583, ar: 0.8098494224800913
Guess:   e: 3.9215019438208834, o: 0.08168271726135921, ar: 0.6535836573034806
MaxCut (7, 7): 6.0
Optimal: e: 4.859096534880548, o: 0.19209726587397577, ar: 0.8098494224800913
Guess:   e: 3.921501943820883, o: 0.08168271726135921, ar: 0.6535836573034804
MaxCut (7, 7): 6.0
Optimal: e: 4.645894564003584, o: 0.18331158137993583, ar: 0.7743157606672639
Guess:   e: 3.729068716035566, o: 0.0790245796722751, ar: 0.6215114526725943
MaxCut (7, 7): 6.0
Optimal: e: 4.859096534880549, o: 0.19209726587397583, ar: 0.8098494224800915
Guess:   e: 3.921501943820883

In [16]:
import QAOA_paper_proxy as qpp
from QAOA_simulator import get_expectation, get_simulator, get_overlap
import qokit.maxcut as mc
import Graph_util

N = 10
G = Graph_util.erdos_renyi(N, N*(N-1)//4)
p = 2
init_gamma, init_beta = np.full((2, p), 0.1)

result = qpp.QAOA_paper_proxy_run(G.number_of_edges(), N, p, init_gamma, init_beta)
print(result)

gamma = result["gamma"]
beta = result["beta"]
sim = get_simulator(N, mc.get_maxcut_terms(G))
print(max(sim.get_cost_diagonal()))
print(get_expectation(N, mc.get_maxcut_terms(G), gamma, beta, sim))
print(get_overlap(N, mc.get_maxcut_terms(G), gamma, beta, sim))

{'gamma': array([-0.24278705, -0.28953577]), 'beta': array([-0.33889322, -0.1402922 ]), 'expectation': 13.781234466694547, 'runtime': 1.4044764041900635, 'num_QAOA_calls': 164, 'classical_opt_success': True, 'scipy_opt_message': 'Optimization terminated successfully.'}
16.0
12.676717441494127
0.04162939174146642


In [10]:
N = 16
num_edges = 100
guess = regr.predict(np.array([[N, num_edges]]))
h_guess, hc_guess, l_guess, r_guess = guess[0][0], guess[0][1], guess[0][2], guess[0][3]
res_guess = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h_guess, hc_tweak_add=hc_guess, l_tweak_mul=l_guess, r_tweak_mul=r_guess)
gamma_guess = res_guess["gamma"]
beta_guess = res_guess["beta"]
for _ in range(5):
    G = Graph_util.erdos_renyi(N, num_edges)
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)
    max_cut = max(sim.get_cost_diagonal())
    guess_e = get_expectation(N, ising_model, gamma_guess, beta_guess, sim)
    guess_approx_ratio = guess_e / max_cut
    print(f"MaxCut ({N}, {num_edges}): {max_cut}")
    print(f"Guess:   e: {guess_e}, o: {get_overlap(N, ising_model, gamma_guess, beta_guess, sim)}, ar: {guess_approx_ratio}")

MaxCut (16, 100): 60.0
Guess:   e: 50.76350440205897, o: 0.00021556804961488012, ar: 0.8460584067009829
MaxCut (16, 100): 61.0
Guess:   e: 50.766141351935424, o: 0.00017873518793295887, ar: 0.8322318254415644
MaxCut (16, 100): 61.0
Guess:   e: 50.7635088917827, o: 4.469442237754685e-05, ar: 0.8321886703570934
MaxCut (16, 100): 59.0
Guess:   e: 50.75652078338172, o: 0.0007484131453241823, ar: 0.8602800132776562
MaxCut (16, 100): 60.0
Guess:   e: 50.76088433992945, o: 4.310899936791724e-05, ar: 0.8460147389988241


In [16]:
from QAOA_simulator import QAOA_run

N = 16
num_edges = 100
guess = regr.predict(np.array([[N, num_edges]]))
h_guess, hc_guess, l_guess, r_guess = guess[0][0], guess[0][1], guess[0][2], guess[0][3]
res_guess = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h_guess, hc_tweak_add=hc_guess, l_tweak_mul=l_guess, r_tweak_mul=r_guess)
gamma_guess = res_guess["gamma"]
beta_guess = res_guess["beta"]
for _ in range(2):
    G = Graph_util.erdos_renyi(N, num_edges)
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)
    max_cut = max(sim.get_cost_diagonal())
    print(f"MaxCut: {max_cut}")
    result_warm = QAOA_run(ising_model, N, p, gamma_guess, beta_guess, "Nelder-Mead")
    print(result_warm)
    print()
    init_gamma, init_beta = np.random.rand(2, p)
    result_cold = QAOA_run(ising_model, N, p, init_gamma, init_beta, "Nelder-Mead")
    print(result_cold)
    print()

MaxCut: 59.0
{'gamma': array([ 0.50725279, -0.07030606]), 'beta': array([-2.32236035,  2.58848055]), 'state': array([-0.00115064-0.00307481j, -0.00093144+0.00166761j,
        0.00011638+0.00071606j, ...,  0.00011638+0.00071606j,
       -0.00093144+0.00166761j, -0.00115064-0.00307481j]), 'expectation': 54.46964162861684, 'overlap': 0.011245020611886042, 'runtime': 177.243812084198, 'num_QAOA_calls': 357, 'classical_opt_success': True, 'scipy_opt_message': 'Optimization terminated successfully.'}

{'gamma': array([ 0.3484321 , -1.42977983]), 'beta': array([0.16365139, 1.44104156]), 'state': array([-0.00138694+2.14008765e-05j,  0.00032111+1.77718655e-03j,
       -0.00139223+5.05010768e-04j, ..., -0.00139223+5.05010768e-04j,
        0.00032111+1.77718655e-03j, -0.00138694+2.14008765e-05j]), 'expectation': 54.497738676845046, 'overlap': 0.011980109491886657, 'runtime': 114.30378150939941, 'num_QAOA_calls': 227, 'classical_opt_success': True, 'scipy_opt_message': 'Optimization terminated suc